# Mini Project 2

**Due: Monday 4 Nov 2019 11:59PM**

This project is an opportunity for you to practice and review the concepts we have seen in the first part of the course. The project is designed to meet the following learning outcomes:

1. Make you more comfortable with static code analysis and transformations.
2. Give you hands-on experience with the different types of embeddings and their tradeoffs from a design perspective.
3. Show you a snippet of when and how to use instrumentation and aspect-oriented programming.
4. (Bonus) Demonstrate how to use BNF notation practically for parsing and interpretation.

**Submission: ** This mini-project consists of two problems with a bonus part. Please submit your solutions through this Google Form: https://forms.gle/kFMJT5gnsEpeBFiS9 **The solution of each problem should be submitted as a different Zip file containing your solution files. Your files should have the format described in each problem.**

**Programming Languages:** Each problem has a list of recommended programming languages to use. You can choose any programming language to carry out this homework, including ones not on the list, this may make certain parts harder to solve.

**Collaboration:** You should solve the problems individually. You are allowed to discuss the problems and your ideas with classmates, but you are prohibited from sharing or copying your solution code (in whole or part). You are free to use any ideas or code you find online.

**Grading:** The grade assigned to each part is displayed next to its description. You will receive full credit if your solution meets all the requirements in the description and passes our correctness tests. You will receive partial credit for parts that are not completely solved or that fail some tests. In addition, we reserve the right to add extra credit for code style and modularity, up to a maximum of 10 extra points.

The problems here are inspired by this Quora answer: https://www.quora.com/What-are-the-top-5-features-missing-from-our-current-enterprise-programming-languages/answer/Tim-Mensch. Please read the answer carefully to see how your solutions can be useful in the real world!

Feel free to ask any questions on Piazza. Remember, Piazza participation = extra credits!

## Problem 1 - Code Analysis and Transformation: Null-Safe Variables (80 points)

**Recommended languages:** python and javascript. Java works but will make some things a bit harder.

NullPointerExceptions and null (or None in python) values in general can be a big problem for programmers, especially in large code bases. Programmers frequently make mental assumptions about their code. One of the most popular such invariants is assuming that a variable will never be given a null value. However, such variables may become null due to some unexpected behavior or bug. Mentally enforcing that variables never get a null value can be hard to do, since variables are frequently assigned values returned by function or user input, or may be assigned values from pieces of the code written by different developers, etc.

It would be nice to have language-level support to enforce that some variables cannot be null. This is supported at the type level in some statically typed languages. However, this is still unsupported by many programming languages, especially dynamically-typed languages (like Python or JavaScript).

In this problem, you will create a static analyzer in your language of choice, that will read code in that language, and determine if certain *protected* variables get assigned nulls or not. Because we will be dealing with dynamic language, we will not enforce this at the type level, but rather by analyzing the code and looking for a naming convention. **Variables that start with \_n\_ in the input code are considered protected**.

**Simplification:** To simplify the problem, we will only consider a restricted subset of the language. **You need to only support** the simplest constructs of the language: variable definitions and assignments, function definitions and calls, and return statements. **You do not need to support** complicated assignments (e.g. unpacking tuples in python or defining several variables in a single statement), conditionals or loop constructs, anonymous or nested functions, keyword arguments, or anything else.

**Submission Instructions:** The problem is made out of 3 parts shown below. The main entry point to each solution should be in a separate file named: part1, part2, and part3 respectively. It is recommended you split your solutions into other files as well, and use functions and classes to make your code as clean as possible.

### Part 1 (a) - Plain Assignments (15 points)

You must implement functionality to parse input code and check whether it has illegal (i.e. null) assignments to protected variables. Your solution file should be run from the command line. It receives the path for the source code file to analyze as a command line argument. Your code should read the content of that file and parse it (using ast, babel, or a similar library in your language of choice). Here is an example main function in python:

In [1]:
import ast, sys

if __name__ == '__main__':
    filename = sys.argv[1]
    with open(filename, 'r') as code_file:
        code = code_file.read()
        tree = ast.parse(code)
        solution(tree)  # your solution function

Your solution must look for variables in the source code, whose names start with '\_n\_'. Any variable with such a name is considered protected, and should not be allowed to get a null value. Your solution must check that this is the case. If your solution detects that an illegal assignment occurs, it must output an error with the line number in the source code where the illegal assignment occured. If several illegal assignments exist, your code should display all of them in order.

In particular, you need to detect and display errors for two types of illegal assignments:
1. If a protected variable is directly assigned null: this is bad and should trigger an error.
2. If a protected variable is assigned a non-protected variable: the non-protected variable may or may not be null, depending on the program execution and input values, in general, it is undecidable to determine if such a variable will be null or not. We will be conservative and display errors for these assignments.

**Remark:** You must perform these checks on all assignments in the code, including ones inside function definitions.

If you are using Python, you can test your implementation of this part by running it on these input files: [test1.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test1.py), [test2.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test2.py). If you are using a different language, you should write your own similar tests.

### Part 1 (b) - Functions (30 points)

The next step is to support protection when it comes to function calls. Function calls and definition cause implicit assignments that can break our protection of variables. Similar to the previous part, we will consider functions whose name start with \_n\_ to be protected: they should not return null. Function parameters with names that start with \_n\_ are also protected: that function cannot be called with a matching null argument.

In particular, you need to detect and display errors for these cases:
1. If a protected variable is assigned the value returned by a function call, and the function name does not start with \_n\_, we will be conservative and display an error.
2. For every function call (even functions with names that does not start with \_n\_), if any parameter in the function definition whose name starts with \_n\_ is passed an unprotected variable or null, an error should be displayed.
3. For every function whose name start with \_n\_, the function cannot return null, an unprotected variable, or a call to an unprotected function.

Here are some useful hints:

**Hint 1**: the second case can only be checked after the protected parameters in the function definition have been identified.

Therefore, Your solution must perform two passes/walks over the parsed code. The first looks for all function definitions, find the protected parameters in them (the ones that start with \_n\_), and store inside your code a mapping between the function name and the position of all protected parameters of that function. 

The second pass can then look at that mapping to determine which parameters are protected, whenever it encounters a function call, it should also perform the functionality from part 1-a.

**Hint 2**: for the third case, it is not enough to only look for return statements inside the function definition. Depending on your language (e.g. in python or javascript), a function without a return statement automatically return null.

**Hint 3**: there is a lot of similarities between cases here and cases from part 1-a. Make sure you organize your code into functions, and then call these functions when needed, instead of writing the same code several times. For example, code to check that a statement is a function call, and the function name starts with \_n\_ can be reused for cases 1 and 3.

If you are using Python, you can test your implementation of this part by running it on this input file: [test3.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test3.py).

### Part 2 - Casting (10 points)

Unfortuntely, our checks from part 1 are too restrictive for programmers to actually be able to use them productively. For example, programmers can never use them to ensure that variables are both not null and are assigned user input or function values from libraries.

In this part, you must create a function called 'null\_cast' in a separate file. This function should take exactly one parameter, and check if that parameter is null. If it is, your function must raise an error. Otherwise, it should return the parameter.

You must modify your solution from part 1 to allow a special case: a protected variable (or function parameter) is allowed to be assigned the result of null\_cast.

**Note:** Our checks now are more flexible and usable. Programmers can use null_cast to do more interesting things. However, null\_cast may raise exceptions. At first glance this seems equivalent to plain programming without our solution, where null values can cause NullPointerExceptions. This is not the case, programmers will know to surround null\_cast with a try catch statement when using it, additionally, it reduces the number of statements where exceptions may occur from essentially any statement involving a variable to only null\_cast, which will be used infrequently by the programmers.

Note that this null\_cast function is a shallow embedding. It allows us to modify the semantics of the language combined with our null checker, while being coded in the statements of the language itself.

You can find an example for testing at this file: [test4.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test4.py).

### Part 3 - Automatic Casting (25 points)

Typing null\_cast frequently or having it in the code in several locations may be distracting to the programmer. We can automatically make calls to null\_cast in locations of the code that are reasonable.

Adding null\_cast calls around every variable assignment is unreasonable. It will slow down the code a lot, and it will remove a lot of control from the programmer. It also makes our checks from part 1 meaningless. Instead, we will only do this automatically for two cases: function calls and return statements, we will do so using instrumentation and aspect oriented programming.

You need to write your code in a separate file with a main function similar to part 1, that receives the input file path as a command line argument. Your main function will perform the instrumentation. Additionally, your file must contain a function called 'null\_weave' that will perform the aspect-oriented solution.

**Instrumentation:** Your main function must parse the input code, and look for all function definitions. Your code must add checks to the start of each function, that check that each protected parameter is not null, and produce an error if it was. If a function has no protected parameter (i.e. all its parameters have names that do not start with \_n\_), the function code should not be modified.

**Aspect-Oriented Programming:** If you are using a language with aspect-oriented support (e.g. python and aspectlib), you must create an aspect that is applied to every function call (even unprotected ones). Your aspects proceeds to call the original function, and then check if the function name starts with \_n\_, and that the return value is null, and raises an error if that was the case. You should create a function 'null\_weave' that attaches the aspect to function calls whenever it is called.

You can find an example for testing at this file: [test5.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test5.py).

**If you are using a language that does not support aspect oriented programming nicely (e.g. Javascript), you should use instrumentation to solve both tasks.**

## Problem 2 - Deep Embedding: Function Call Optimization (20 points + 40 bonus)

**Recommended Languages:** It is recommended that you use Python for this problem.

In this problem, we will use Deep Embedding to optimize repeated calls to the same function. This problem is made out of several parts. First, we will optimize function calls by caching their results, which works if a function is called several times with exactly the same parameter. Later, we will optimize calls that use the same first parameter, but may have a different parameter.

### Part 1 - Automatic Memoization (10 points)

[Memoization](https://en.wikipedia.org/wiki/Memoization) is a standard technique for optimizing divide-and-conquer recursive algorithms. Certain recursive algorithms make several recursive calls, some of which may be on exactly the same parameter. You only need to compute the answer to one of these calls, store that in a cache/table/dict, and return that value directly for future calls. You can read more about [dynamic programming](https://www.geeksforgeeks.org/dynamic-programming/).

In this part, you have to create a function called 'memoize' inside a file 'part1.py'. This function takes another function and its arguments as parameters. The memoize function will maintain some table that caches the output of all underlying functions. If the table does not have the output for the given function and parameters, memoize should run the function on the parameters and save that result.

This table can be implemented via a global python dictionary, where the keys to that table consist of function names and arguments, and the values in the table will be the output of running that function on these arguments. **For simplification you can assume that all function parameters are numbers.**

Below is a more detailed description of the function. Look at this file for some tests [test1.py](https://github.com/KinanBab/CS591L1/project-2/problem2/test1.py).

In [5]:
def memoize(f, *args, **kwargs):
    # check if f(*args, **kwargs) is already computed and stored
    # if it is, return the output directly,
    # otherwise, call f(*args, **kwargs) and save its result, and return it.
    pass

### Part 2 - Automatic Wrapping (10 points)

Looking at the previous test files, you see that every function call the user make, has to be wrapper in a call to memoize. This is annoying since users will have to remember to do that manually, and they have to modify their code in order to do it.

In this part, you should write another function 'memoize_aspect' in a different file 'part2.py'. The implementation of memoize_aspect is similar to the above memoize. However, it must be compatible with aspectlib. Look at the programming paradigms lecture notes for some examples. You will need to use bind=True parameter to make the memoize function receive the function being called, you also need to use the appropriate advice from aspectlib (e.g. Proceed).

You should use weave API of aspectlib to attach memoize_aspect to all function calls.

### Bonus - Part 3 - Expression Tree Partial Optimization (40 points)

This part is a bonus part. Look at the deep embedding lecture notes for some help.

What we want to do now is to be able to optimize function calls, even if some of the parameters are different.

In particular, and for simplification, consider only the case where we have a function that takes two parameters, the first is a matrix (2D list), and the second is a number. The program will make several calls to the function, each call uses the same matrix, but has a different second argument.

We want to optimize all the calls after the first one. In particular, the pieces of the function code that operates only on the first parameter need to be executed only once, and their results can be saved and reused in the other calls. The difficulty is that our optimization code should somehow dynamically open the function up, and identify the expressions that can be optimized, and the ones that cannot be. We can do this via (somewhat) Lazy evaluation.

A use case of this is solving systems of linear equations. These are usually represented as a matrix (containing coefficients of equations) and a vector containing values for every equation. You can see an example [here](https://en.wikipedia.org/wiki/System_of_linear_equations#Matrix_equation).

Certain popular algorithms for solving such systems works by first perfoming an [LU factorization](https://math.stackexchange.com/questions/1162992/how-to-use-lu-decomposition-to-solve-ax-b) step on the matrix (very expensive, O(n^3)), and then performig cheaper forward and backwards passes on the factorized matrices and the vector.

I provide a simplified version of the example above. Instead of receiving as parameters both a matrix and a vector, it receives a matrix and a number. The function performs LU factorization, then does some summation operation on it and the give number. This prototype is available at [test2.py](https://github.com/KinanBab/CS591L1/project-2/problem1/test2.py). You can use it to test your implementation for this part.

You will implement a function 'optimize' in a new file 'part3.py'. The optimize call receives the desired function to run and the two parameters, it performs the optimization described above, and returns the result.

In [7]:
def optimize(f, arg1, arg2):
    # check if f and arg1 has already been optimized, if it has, evaluate it for arg2
    # otherwise, evaluate f(arg1, arg2) and store the optimized expression tree for arg1.
    pass

Here are some instructions on how to achieve this. These are only one possible way of doing it. Other approaches are acceptable too. We are happy to discuss any other ideas via Piazza or in person:
1. Create a Lazy class similar to the one from the Deep Embedding lecture notes. You need to override all operations that are used by the test2.py file: +, -, \*, // (\_\_floordiv\_\_).
2. Every single operation overriden in the class should return a new Lazy instance representing that operation, similar to the Lazy class from class.
3. The constructor of the Lazy class should take the value (similar to the notes) as well as a label parameter, which specifies whether this instance belongs to a part of the first or second argument (or neither).
4. Finally, the class should have two functions: eval1(self) and eval2(self, val). The first evaluates all expressions that do not belong to the second parameter. The second evaluates the ones that belong to the second parameter, where the value of that parameter is given via argument val.
5. Your optimize function should check if f has been optimized with arg1 already, if it has, it can use eval2 on the optimized expression tree, and pass arg2 to it to get the final result.
6. If the function and arg1 were not optimized before, the optimize function first turns arg1 and arg2 into Lazy instances (a matrix/list is turned to lazy by making every element in it lazy), when creating Lazy objects, the function should pass the appropriate label to identify if it is part of the first or second argument, then calling the function passing it the lazy arguments. The returned expression tree is evaluated on the first argument using eval1, then stored for further use, then evaluated on the second argument using <expr>.eval2(arg2), and the result is returned to the user.
    
Hint: Make sure that any calls to eval1 and eval2 creates copies of the expressions but never modify the actual expression tree itself, so that stored copies of the tree are not affected.